In [16]:
import sagemaker
from sagemaker import get_execution_role
import boto3
import json
import pandas as pd
from sagemaker.serializers import CSVSerializer
import numpy as np

In [17]:
sess = sagemaker.Session()
region = boto3.Session().region_name
bucket = "mysagemakertutorial"
data_prefix = "sagemaker/data"  
model_prefix = "sagemaker/model"
role = get_execution_role()

In [18]:
from sagemaker.amazon.amazon_estimator import get_image_uri

#### Get the built-in xgboost container image in Sagemaker to host our model
container = get_image_uri(boto3.Session().region_name, "xgboost", "0.90-1")
# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
xgboost_container = sagemaker.image_uris.retrieve("xgboost", region, "latest")
xgboost_container

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


'433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest'

In [19]:
model_name = 'xgboost-2021-12-22-16-10-49-300'
model_url = 'https://mysagemakertutorial.s3.us-west-2.amazonaws.com/sagemaker/model/xgboost-2021-12-22-16-10-49-300/output/model.tar.gz'

In [20]:
%%time
model = sagemaker.Model(
    image_uri=xgboost_container, model_data=model_url, role=role, sagemaker_session=sess
)
model.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")

predictor = sagemaker.Predictor(
    endpoint_name=model.endpoint_name,
    sagemaker_session=sess,
    serializer=CSVSerializer()
)

-------!CPU times: user 128 ms, sys: 1.71 ms, total: 130 ms
Wall time: 3min 31s


In [21]:
data_key = 'test/test.csv'
data_location = 's3://{}/{}/{}'.format(bucket, data_prefix,data_key)
print(data_location)
test_data = pd.read_csv(data_location, header = None)

s3://mysagemakertutorial/sagemaker/data/test/test.csv


In [22]:
test_data.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.42450,0.45040,0.24300,0.3613,0.08758
1,15.85,23.95,103.70,782.7,0.08401,0.10020,0.09938,0.05364,0.1847,0.05338,...,16.84,27.66,112.00,876.5,0.1131,0.19240,0.23220,0.11190,0.2809,0.06287
2,13.54,14.36,87.46,566.3,0.09779,0.08129,0.06664,0.04781,0.1885,0.05766,...,15.11,19.26,99.70,711.2,0.1440,0.17730,0.23900,0.12880,0.2977,0.07259
3,17.14,16.40,116.00,912.7,0.11860,0.22760,0.22290,0.14010,0.3040,0.07413,...,22.25,21.40,152.40,1461.0,0.1545,0.39490,0.38530,0.25500,0.4066,0.10590
4,11.94,18.24,75.71,437.6,0.08261,0.04751,0.01972,0.01349,0.1868,0.06110,...,13.10,21.33,83.67,527.2,0.1144,0.08906,0.09203,0.06296,0.2785,0.07408


In [23]:
test_array = np.array(test_data)

In [24]:
%%time
predictions = predictor.predict(test_array)
print(predictions)

b'0.8857102990150452,0.4335191249847412,0.14776110649108887,0.8857102990150452,0.09307307004928589,0.09307307004928589,0.09307307004928589,0.3406851291656494,0.6615563631057739,0.09307307004928589,0.09307307004928589,0.09307307004928589,0.09307307004928589,0.2544648051261902,0.8857102990150452,0.7025442123413086,0.09307307004928589,0.09307307004928589,0.09307307004928589,0.09307307004928589,0.09307307004928589,0.09307307004928589,0.8229480981826782,0.7543903589248657,0.09307307004928589,0.6221315860748291,0.09307307004928589,0.8857102990150452,0.8857102990150452,0.4335191249847412,0.8857102990150452,0.09307307004928589,0.09307307004928589,0.09307307004928589,0.8857102990150452,0.09307307004928589,0.8857102990150452,0.09307307004928589,0.09307307004928589,0.8857102990150452,0.8857102990150452,0.09307307004928589,0.09307307004928589,0.09307307004928589,0.09307307004928589,0.09307307004928589,0.09307307004928589,0.09307307004928589,0.8857102990150452,0.3406851291656494,0.44106248021125793